In [ ]:
#from https://stackoverflow.com/a/31434967
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

<h1> Bank Marketing Campaign </h1>

We are provided with data on a recent marketing campaign conducted by a bank to encourage individuals to sign up for a term deposit. 

The challenge here is to build a model to be able to predict whether or not a given individual will sign up for a term subscription if contacted as part of this marketing campaign. 

This notebook will explore the dataset provided and use machine learning and statistical inferencing to build a predictive model. At the end, based on the insights generated by the model, we also provide suggestions on types of individuals to approach to increase the effectiveness of this campaign.

Kindly note that the approaches used in this model is based on https://www.kaggle.com/janiobachmann/bank-marketing-campaign-opening-a-term-deposit. Additional resources are included at the end of this notebook.

## 1. Import packages and data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import plotly.graph_objs as go
import squarify
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

seed=42

In [ ]:
df_raw = pd.read_csv(r'DATA\bank.csv')
df_full = df_raw.copy()

## 2. Understand data

Copied from this kaggle notebook: https://www.kaggle.com/janiobachmann/bank-marketing-campaign-opening-a-term-deposit/notebook

1 - **age:** (numeric)<br>
2 - **job:** type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - **marital:** marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - **education:** (categorical: primary, secondary, tertiary and unknown)<br>
5 - **default:** has credit in default? (categorical: 'no','yes','unknown')<br>
6 - **housing:** has housing loan? (categorical: 'no','yes','unknown')<br>
7 - **loan:** has personal loan? (categorical: 'no','yes','unknown')<br>
8 - **balance:** Balance of the individual.<br>
9 - **month:** last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
10 - **day:** last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>
11 - **duration:** last contact duration, in seconds (numeric).<br>
12 - **campaign:** number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
13 - **pdays:** number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>
14 - **previous:** number of contacts performed before this campaign and for this client (numeric)<br>
15 - **poutcome:** outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br>
16 - **contact:** contact communication type (categorical: 'cellular','telephone') <br>

Output variable (desired target):<br>
17 - **y** - has the client subscribed a term deposit? (binary: 'yes','no')

In [ ]:
df_full.head()

In [ ]:
df_full.shape

No missing values

In [ ]:
df_full.info()

In [ ]:
plt.figure(figsize=(16,8))
labels ="Did not deposit", "Deposited"
colors = ['c', 'm']
plt.suptitle('Information on Term Suscriptions', fontsize=20)
plt.pie(df_full["deposit"].value_counts(),autopct='%1.2f%%', labels=labels, colors=colors)
plt.ylabel('% Deposited', fontsize=14)

Distribution of numerical data

In [ ]:
df_full.hist(bins=20, figsize=(16,8))
plt.show()

Hunch is that jobs impact likelihood of subscribing to term deposit.

Let's remove "unknown" job.

In [ ]:
df_full = df_full.drop(df_full.loc[df_full.job=="unknown"].index)

And replace "admin" with management since fairly similar.

In [ ]:
df_full.job = df_full.job.replace("admin.", "management")

In [ ]:
df_full.job.value_counts()

Let's see correlations

#### Correlations

In [ ]:
#need to one-hot encode our target variable
from sklearn.preprocessing import LabelEncoder

fig=plt.figure(figsize=(16,8))
df_full.deposit = LabelEncoder().fit_transform(df_full.deposit)

#we can only use numerical features for correlation analysis
numeric_df_full = df_full.select_dtypes(exclude="object")

cor_diagram = numeric_df_full.corr()

sns.heatmap(cor_diagram, cbar=True)
plt.title("Correlation matrix")
plt.show()

#### Duration vs deposit

From here, we see campaign last contact duration has the highest correlation with whether or not a person subscribes for a deposit.

In [ ]:
df_full["duration_status"] = np.nan

avg_duration = df_full["duration"].mean()

df_full["duration_status"] = df_full.apply(lambda row: "below_average" if row.duration<avg_duration else "above_average",axis=1 )

yes_no_by_durationstatus = pd.crosstab(df_full['duration_status'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_durationstatus

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_durationstatus.plot(kind='bar', stacked=False)
plt.title("The Impact of Duration vs Opening a Deposit", fontsize=18)
plt.xlabel("Duration Status", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

From here, we see that if you had an above average duration in last campaign, and you were contacted again, majority would subscribe. Conversely, if you had a short duration of contact in last campaign, and you were contacted again, majority would not subscribe to a deposit. This is possibly a sign of interest and/or rapport. Ie customers we spent more time talking to were more open to subscribing.

#### Balance vs deposit

In [ ]:
df_full['temp_balance'] = np.ceil(df_full["balance"]/10000)*10000
yes_no_by_temp_balance = pd.crosstab(df_full['temp_balance'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_balance

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_balance.plot(kind='bar', stacked=False)
plt.title("The Impact of Balance vs Opening a Deposit", fontsize=18)
plt.xlabel("Balance", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

From the above, we see that generally those who did not subscribe to a term deposit tend to have lower bank balances. 

Note: 0 here means they had negative balances ie they were in overdraft.

#### Age vs deposit

In [ ]:
df_full['temp_age'] = np.ceil(df_full["age"]/10)*10
yes_no_by_temp_age = pd.crosstab(df_full['temp_age'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_age

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_age.plot(kind='bar', stacked=False)
plt.title("The Impact of Age vs Opening a Deposit", fontsize=18)
plt.xlabel("Age", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

From the above, if you were very young or a senior citizen, you were more likely to subscribe to a bank deposit.

#### Pdays vs deposit

In [ ]:
df_full['temp_pdays_bucket'] = df_full.apply(lambda row: 0 if row['pdays']<=0 else 1 if row['pdays']<=30 else 2 if row['pdays']<=60 else 3 if row['pdays']<=90 else 6 if row['pdays']<=180 else 12 if row['pdays']<=365 else 24 if row['pdays']<=730 else 25,axis=1)

In [ ]:
df_full['temp_pdays_bucket'].value_counts()

In [ ]:
yes_no_by_temp_pdays = pd.crosstab(df_full['temp_pdays_bucket'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_pdays

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_pdays.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of Pdays (months) vs Opening a Deposit", fontsize=18)
plt.xlabel("Pdays (months)", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

Not clear but appears to suggest that if someone was recently contacted, and if they were contacted again this time, they were more likely to decline the offer to subscribe. Seems to suggest that the longer someone has not been contacted, the more likely they are to subscribe.

Note: 0 means they were just contacted. 25 means more than 24 months since last contact

#### Campaign vs deposit

In [ ]:
df_full['campaign'].hist()

In [ ]:
df_full['temp_campaign'] = df_full.apply(lambda row: 11 if row['campaign']>=11 else row['campaign'],axis=1)

In [ ]:
yes_no_by_temp_campaign = pd.crosstab(df_full['temp_campaign'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_campaign

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_campaign.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of Number of times contacted this campaign vs Opening a Deposit", fontsize=18)
plt.xlabel("Number of times contacted", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

From the above, repeat contacts make it even less likely for someone to subscribe.

#### Previous outcome vs deposit

In [ ]:
df_full['poutcome'].hist()

In [ ]:
yes_no_by_poutcome = pd.crosstab(df_full['poutcome'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_poutcome

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_poutcome.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of previous campaign vs Opening a Deposit", fontsize=18)
plt.xlabel("Previous campaign outcome", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

If previous outcome was succesful, you were overwhelmingly more likely to subscribe.

#### Number of times contacted in previous campaign vs outcome of this campaign

In [ ]:
df_full['previous'].hist()

In [ ]:
df_full['temp_previous'] = df_full.apply(lambda row: 11 if row['previous']>=11 else row['previous'],axis=1)

In [ ]:
yes_no_by_temp_previous = pd.crosstab(df_full['temp_previous'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_previous

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_previous.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of Number of times contacted previous campaign vs Opening a Deposit", fontsize=18)
plt.xlabel("Number of times contacted", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

Interestingly, the more you contacted someone in previous campaign, the more likely they were to subscribe to a term deposit.
Conversely, if they had never heard from you before (not contacted from previous campaign), they were less likely to subscribe this time.

Note that there may be some correlations between 'duration', 'previous contact times' and 'previous outcome'. As we are likely to use a tree-based model, multicollinearity is not an issue but we will be testing this assumption in order to flush out what exactly are key predictors.

For now, let's visualise these features.

#### Duration vs deposit, given success of previous campaign

In [ ]:
df_full["duration_status"] = np.nan

avg_duration = df_full["duration"].mean()

df_full["duration_status"] = df_full.apply(lambda row: "below_average" if row.duration<avg_duration else "above_average",axis=1 )

yes_no_by_durationstatus = pd.crosstab(df_full[df_full['poutcome']=="success"]['duration_status'], df_full[df_full['poutcome']=="success"]['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_durationstatus

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_durationstatus.plot(kind='bar', stacked=False)
plt.title("The Impact of Duration vs Opening a Deposit is previous campaign was successful", fontsize=18)
plt.xlabel("Duration Status", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()


In [ ]:


df_full["duration_status"] = np.nan

avg_duration = df_full["duration"].mean()

df_full["duration_status"] = df_full.apply(lambda row: "below_average" if row.duration<avg_duration else "above_average",axis=1 )

yes_no_by_durationstatus = pd.crosstab(df_full[df_full['poutcome']=="failure"]['duration_status'], df_full[df_full['poutcome']=="failure"]['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_durationstatus



In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_durationstatus.plot(kind='bar', stacked=False)
plt.title("The Impact of Duration vs Opening a Deposit is previous campaign was a failure", fontsize=18)
plt.xlabel("Duration Status", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

From here, it suggests that if your previous outcome was not a success and if you spent a lot of time in contact with them in that failed campaign, it is unlikely you would see much success in a repeat campaign.

(From earlier sections, we already know that prior campaign success seems to relate with a repeat success!)

#### Number of days contacted in previous campaign vs success in this campaign, given success of previous campaign

In [ ]:
df_full['temp_previous'] = df_full.apply(lambda row: 11 if row['previous']>=11 else row['previous'],axis=1)

In [ ]:
yes_no_by_temp_previous = pd.crosstab(df_full[df_full['poutcome']=="success"]['temp_previous'], df_full[df_full['poutcome']=="success"]['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_previous

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_previous.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of Number of times contacted previous campaign vs Opening a Deposit if previous campaign successful", fontsize=18)
plt.xlabel("Number of times contacted", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

In [ ]:
yes_no_by_temp_previous = pd.crosstab(df_full[df_full['poutcome']=="failure"]['temp_previous'], df_full[df_full['poutcome']=="failure"]['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_previous

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_previous.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of Number of times contacted previous campaign vs Opening a Deposit if previous campaign not successful", fontsize=18)
plt.xlabel("Number of times contacted", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

From the above, if the previous campaign failed, it is not exactly clear whether or not an existing 'rapport' (by way of having contacted them a few times before) is actually helpful in convincing them to sign up this time.

(From earlier sections, we already know that prior campaign success seems to relate with a repeat success!)

#### Days since last campaign vs success of this campaign, given success of previous campaign

In [ ]:
df_full['temp_pdays_bucket'] = df_full.apply(lambda row: 0 if row['pdays']<=0 else 1 if row['pdays']<=30 else 2 if row['pdays']<=60 else 3 if row['pdays']<=90 else 6 if row['pdays']<=180 else 12 if row['pdays']<=365 else 24 if row['pdays']<=730 else 25,axis=1)

In [ ]:
yes_no_by_temp_pdays = pd.crosstab(df_full[df_full['poutcome']=="success"]['temp_pdays_bucket'], df_full[df_full['poutcome']=="success"]['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_pdays

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_pdays.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of Pdays (months) vs Opening a Deposit if previous campaign was successful", fontsize=18)
plt.xlabel("Pdays (months)", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

In [ ]:
yes_no_by_temp_pdays = pd.crosstab(df_full[df_full['poutcome']=="failure"]['temp_pdays_bucket'], df_full[df_full['poutcome']=="failure"]['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_pdays

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_pdays.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of Pdays (months) vs Opening a Deposit if previous campaign was unsuccessful", fontsize=18)
plt.xlabel("Pdays (months)", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

#### Housing loan vs deposit

In [ ]:
yes_no_by_temp_housing = pd.crosstab(df_full['housing'], df_full['deposit']).apply(lambda row: round(row/row.sum(),4) * 100, axis=1)
yes_no_by_temp_housing

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
ax = yes_no_by_temp_housing.plot(kind='bar', stacked=False)#, order=['0m','1m','2m', '3m', '6m','12m', '24m', '>24m'])
plt.title("The Impact of having an existing housing loan vs Opening a Deposit", fontsize=18)
plt.xlabel("Has housing loan", fontsize=18);
plt.ylabel("Percentage (%)", fontsize=18)

for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.02, p.get_height() * 1.02))
    

plt.show()

From the above, if you had a housing loan, you were less likely to subscribe to a deposit from this campaign.

## 3. Model data

In [ ]:
df_full = df_raw.copy()

In [ ]:
df_full.head()

In [ ]:
df_full[df_full.age.isnull()]

### Split train - test

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

stratifier = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for training_data_indexes, test_data_indexes in stratifier.split(df_full,df_full.loan):
    stratified_training_data = df_full.loc[training_data_indexes]
    stratified_test_data = df_full.loc[test_data_indexes]
    
    
#create copies
train = stratified_training_data.copy()
test = stratified_test_data.copy()

In [ ]:
df_full

In [ ]:
train.shape

In [ ]:
train

In [ ]:
test.shape

### Build transformation pipeline

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

In [ ]:
train.info()

In [ ]:
numeric_cols = train.select_dtypes(include=np.number).columns.tolist()
#numeric_cols.remove('deposit')

In [ ]:
numeric_cols

In [ ]:
categoric_cols = train.select_dtypes(exclude=np.number).columns.tolist()
#remove target variable, to ensure that we end up with feature set, not label
categoric_cols.remove('deposit')
categoric_cols

In [ ]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [ ]:
# Build pipelines to scale numerical features and encode categorical

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Making pipelines
numerical_pipeline = Pipeline([
    ("select_numeric", DataFrameSelector(numeric_cols)),
    ("std_scaler", StandardScaler()),
])

categorical_pipeline = Pipeline([
    ("select_cat", DataFrameSelector(categoric_cols)),
    ("cat_encoder",  OneHotEncoder(handle_unknown='ignore'))
])

from sklearn.pipeline import FeatureUnion
# https://stackoverflow.com/a/52666039
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("numerical_pipeline", numerical_pipeline),
        ("categorical_pipeline", categorical_pipeline),
    ])

In [ ]:
X_train = preprocess_pipeline.fit_transform(train)
X_train = X_train.toarray()
X_train

In [ ]:
X_train.shape

In [ ]:
categorical_pipeline["cat_encoder"].get_feature_names()

In [ ]:
# do not fit the scaler to test features
X_test = preprocess_pipeline.transform(test)
X_test = X_test.toarray()
X_test

In [ ]:
#instantiatie label from stratified datasets
y_train = train['deposit']
y_test = test['deposit']
y_train.shape

In [ ]:
# encode label
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()
y_train = encode.fit_transform(y_train)
y_test = encode.fit_transform(y_test)

In [ ]:
feature_names = numeric_cols.copy()
feature_names.extend([col for col in categorical_pipeline["cat_encoder"].get_feature_names()])

### Train using cross-val

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, cross_val_predict


dict_classifiers = {
    'logistic_reg': LogisticRegression(),
    'support_vector_clf': SVC(),
    'knn_clf': KNeighborsClassifier(),
    'gradient_boosting_clf': GradientBoostingClassifier(),
    'random_forest_cld': RandomForestClassifier(),
    'naive_bayes_clf': GaussianNB(),
    'xgboost_clf': XGBClassifier()
}

dict_preds = {}

for this_classifier in dict_classifiers.keys():
    this_score = cross_val_score(dict_classifiers[this_classifier], X_train, y_train, cv=5)
    # https://stackoverflow.com/questions/25006369/what-is-sklearn-cross-validation-cross-val-score
    this_mean = this_score.mean()
    # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html
    dict_preds[this_classifier] = cross_val_predict(dict_classifiers[this_classifier], X_train, y_train, cv=5) #get predictions, for ROC curve
    print(this_classifier, this_mean)

On just accuracy evaluation, GB classifier appears to perform the best but only marginally better than an svm.

### Evaluate confusion matrix

Plot confusion matrix to see precision and recall. Note the following:

- TN: Those we identified as not taking up a deposit, and in reality did not
- TP: Those we identfied as taking up a deposit, and in reality did
- FN: Those we identified as not taking up a deposit, but in reality did
- FP: Those we identified as taking up a deposit, but in reality did not


  			
- (top row) true positive, false positive
- (bottom row) false negative, true negative

In [ ]:
from sklearn.metrics import confusion_matrix
#https://machinelearningmastery.com/confusion-matrix-machine-learning/
 
for this_classifier in dict_preds.keys():
    this_pred = dict_preds[this_classifier]
    confusion_results = confusion_matrix(y_train, this_pred)
    print(this_classifier, "\n\n", confusion_results, "\n\n")

From the above, comparing svm, gb and xgboost, unable to see a clear winner. Both have not insignificant false positives and negatives.

To have a single metric, we can plot a precision-recall curve.

### Evaluate precision-recall using AUC in ROC.

https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5

In [ ]:
from sklearn.metrics import roc_auc_score

for this_classifier in dict_preds.keys():
    print('{}: {}'.format(this_classifier, roc_auc_score(y_train, dict_preds[this_classifier])))

Looking at AUC, all three are close but gradient boosting_clf is highest. However, given the recent surge in use of xgboost, let's go with this model for now.

### Tune XG Boost

https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

In [ ]:
import xgboost as xgb

In [ ]:
# https://stackoverflow.com/a/46943417
dtrain = xgb.DMatrix(X_train, label=y_train,feature_names=feature_names)
dtest = xgb.DMatrix(X_test, label=y_test,feature_names=feature_names)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
# https://xgboost.readthedocs.io/en/latest/parameter.html
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

params['eval_metric'] = "auc"
params['verbosity'] = 0

num_boost_round = 999

**Set the number of boosting rounds or trees to build, defined by num_boost_round**

If metric has not improved for n number of rounds (defined by early_stopping_rounds), we print the best number of trees to build.

Note, num_boost_round is highly dependent on other hyperparams. Therefore, we will retune this at the end of our various grid-searches.

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

Best boosting rounds is 32 ie 32 trees to build.

**Cross-validation**

Initial cv to see baseline AUC.

In [ ]:
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'auc'},
    early_stopping_rounds=10
)

cv_results

AUC for validation set is lower than training set, indicating likely overfitting.

**Gridsearch - Parameters max_depth and min_child_weight:**

"These parameters add constraints on the architecture of the trees.
- **max_depth** is the maximum number of nodes allowed from the root to the farthest leaf of a tree. Deeper trees can model more complex relationships by adding more nodes, but as we go deeper, splits become less relevant and are sometimes only due to noise, causing the model to overfit.

- **min_child_weight** is the minimum weight (or number of samples if all samples have a weight of 1) required in order to create a new node in the tree. A smaller min_child_weight allows the algorithm to create children that correspond to fewer samples, thus allowing for more complex trees, but again, more likely to overfit."

Source: https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f



In [ ]:
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(6,9)
    for min_child_weight in range(1,8)
]

In [ ]:
# Define initial best params and AUC
# min_mae = float("Inf") # use this if for MAE
max_auc = 0
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    # Update best AUC
    mean_auc = cv_results['test-auc-mean'].max() 
    # mean_rmse = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

In [ ]:
params['max_depth'] = 6
params['min_child_weight'] = 3

In [ ]:
params

**Gridsearch - Parameters subsample and colsample_bytree:**

"These parameters control the sampling of the dataset that is done at each boosting round.
Instead of using the whole training set every time, we can build a tree on slightly different data at each step, which makes it less likely to overfit to a single sample or feature.

- **subsample** corresponds to the fraction of observations (the rows) to subsample at each step. By default it is set to 1 meaning that we use all rows.

- **colsample_bytree** corresponds to the fraction of features (the columns) to use. By default it is set to 1 meaning that we will use all features."

Source: https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

In [ ]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(1,11)]
    for colsample in [i/10. for i in range(1,11)]
]

In [ ]:
max_auc = 0
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    # Update best AUC
    mean_auc = cv_results['test-auc-mean'].max() 
    # mean_rmse = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = (subsample,colsample)
print("Best params: {}, {}, AUC: {}".format(best_params[0], best_params[1], max_auc))

In [ ]:
params['subsample'] = 0.9
params['colsample_bytree'] = 0.8

**Gridsearch - Parameters ETA:**

"The ETA parameter controls the learning rate. It corresponds to the shrinkage of the weights associated to features after each round, in other words it defines the amount of "correction" we make at each step (remember how each boosting round is correcting the errors of the previous).

In practice, having a lower eta makes our model more robust to overfitting thus, usually, the lower the learning rate, the best. But with a lower eta, we need more boosting rounds, which takes more time to train, sometimes for only marginal improvements."

Source: https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f

In [ ]:
%time
# This can take some time…

min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time
    
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    # Update best AUC
    mean_auc = cv_results['test-auc-mean'].max() 
    # mean_rmse = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-auc-mean'].argmax()
    print("\tAUC {} for {} rounds".format(mean_auc, boost_rounds))
    if mean_auc > max_auc:
        max_auc = mean_auc
        best_params = eta
print("Best params: {}, AUC: {}".format(best_params, max_auc))

In [ ]:
params['eta'] = 0.01
params

**Cross-validation again, to decide on num_boost_rounds**

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

From the above, it looks like we need more rounds to get to the best AUC score on the test set compared to before we did our grid-search but at least we have a higher AUC now.




**Check results on training set**

Now that we know what our best number of boosting trees are needed, let's retrain the model again before saving.

In [ ]:
num_boost_round = model.best_iteration + 1
num_boost_round

In [ ]:
our_best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

In [ ]:
params

In [ ]:
XGBClassifier(num_boost_round=num_boost_round)

In [ ]:
dict_classifiers['xgboost_clf'] = XGBClassifier(colsample_bytree=0.8, eta=0.01,
                      eval_metric='auc', max_depth=6,
                      min_child_weight=3, objective='reg:linear',
                      subsample=0.9,seed=42,num_boost_round=804,verbosity=0)

dict_classifiers

In [ ]:
# accuracy
revised_score = cross_val_score(dict_classifiers['xgboost_clf'], X_train, y_train, cv=5)

revised_score.mean()

In [ ]:
revised_preds = cross_val_predict(dict_classifiers['xgboost_clf'], X_train, y_train, cv=5)

In [ ]:
confusion_results = confusion_matrix(y_train, revised_preds)
confusion_results

In [ ]:
revised_auc = roc_auc_score(y_train, revised_preds)
revised_auc

In [ ]:
our_best_model.save_model("my_model.model")

## 4) Make predictions

https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/

"To make predictions we use the scikit-learn function model.predict().

By default, the predictions made by XGBoost are probabilities. Because this is a binary classification problem, each prediction is the probability of the input pattern belonging to the first class. We can easily convert them to binary class values by rounding them to 0 or 1."

In [ ]:
our_best_predictions = our_best_model.predict(dtest)
final_best_predictions = [abs(round(value)) for value in our_best_predictions]

In [ ]:
confusion_results = confusion_matrix(y_test, final_best_predictions)
confusion_results

In [ ]:
revised_auc = roc_auc_score(y_test, final_best_predictions)
revised_auc

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, final_best_predictions)

From here, we see that the test AUC is not too far from training and validation. This means we are not overfitting.

Overall, acceptable results.

Next, we will see what were key features in helping the model make it's decisions.

## 5) Key features

In [ ]:
from xgboost import plot_importance

In [ ]:
plot_importance(our_best_model)
plt.show()

We see the following features were more important for the boosted decisions trees to model the data:

- 'duration': duration (in miliseconds) of last contact<br>
- 'balance': balance of the individual<br>
- 'age': age of this person<br>
- 'pdays': days since last contact<br>
- 'campaign': number of contacts for this person in this campaign<br>
- 'x6_success': if outcome of the previous marketing campaign was succesful<br>
- 'previous': number of times someone was contacted in previous campaign<br>
- 'x4_housing': does not have a housing loan<br>

Let's check these against a basic logistic regression model to see if these features also hold true as significant.

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.GLM.from_formula("deposit ~ duration + balance + age + pdays + campaign + poutcome + previous + housing ", family=sm.families.Binomial(), data=train)
result = model.fit()
result.summary()

From the above, we actually see that age and pdays (days since last contact) may not necessarily have a significant relationship with the odds of someone subscribing to the term deposit (p-values below 0.05). Slightly different conclusion from our xgboost model.

Next, we also look at multicollineariy between previous campaign number of contacts and durations, with previous campaign success. If they were correlated, this may mean our standard errors from the logistic regression model may be wrong, and this would result in incorrect p-values.

In [ ]:
dummy_df = pd.DataFrame(train[['duration', 'previous', 'poutcome']])
dummy_df['poutcome'] = dummy_df.apply(lambda row: 1 if row['poutcome']=='success' else 0,axis=1)
dummy_df_cor = dummy_df.corr()
pd.DataFrame(np.linalg.inv(dummy_df.corr().values), index = dummy_df_cor.index, columns=dummy_df_cor.columns)

The diagonal values indicate the variance inflation factor (VIF) which is a metric for multicollinearity between two predictors. Values below 5 indicate that is collinearity is absent. As such, the signifance of these features are valid.

## 6) Recommended actions for a marketing strategy

Based on the analysis we conducted in the earlier sections of this notebook as well as the insights generated from machine learning, general guidance would be to avoid people with overdrawn bank balances, existing housing loans, and we pestered and failed with in previous attempts.

More precide tactics would be as follows:

- 'duration' and 'previous': Best to try to avoid individuals we 'pestered' before if our previous campaign failed. However, if we had success with them before, we may see success again regardless of how much we pestered them before.<br>
- 'balance': Perhaps greater campaign success by not approaching individuals with overdrawn bank balances<br>
- 'campaign': Repeatedly pestering someone this campaign will make them less likely to sign up. Therefore, need to make sure we build rapport with minimal repeated attempts.<br>
- 'x6_success': Focus on inidividuals where we had success with in our previous campaign<br>
- 'x4_housing': Focus on individuals without housing loan debts (perhaps indicative of additional capital to use).<br>

Other tactics that may be helpful:
- 'age': Approach more people who are either in their 20's or 60's and above<br>
- 'pdays': Perhaps consider focusing on people we may not have spoken to in a while<br>

## 7) Steps to improve analysis

In future iterations of this analysis, we will likely focus on the following:

- Improve false negative rates. This is potential cash on the table, valuable opportunities we may be missing out on
- Try a neural network to see if any improvements

## 8) Resources:

1. https://www.kaggle.com/janiobachmann/bank-marketing-campaign-opening-a-term-deposit: Main reference notebook for steps to proceed

2. https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f: Tips to tune xgboost

3. https://stackoverflow.com/a/31434967: Toggle code button

4. https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html: Logic of cross-val predictions

5. https://machinelearningmastery.com/confusion-matrix-machine-learning/: Code confusion matrix

6. https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5: Explanation of AUC for ROC

7. https://xgboost.readthedocs.io/en/latest/parameter.html: XGBoost parameters

8. https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/: Converting XGBoost Classifier output to binary

9. https://stackoverflow.com/questions/25006369/what-is-sklearn-cross-validation-cross-val-score: Logic of cross-val score

10. https://towardsdatascience.com/multicollinearity-in-data-science-c5f6c0fe6edf: Explanation of variance inflation factor

11. https://stackoverflow.com/a/51463149: Code for variance inflation factor